In [8]:
## pip install pdfplumber

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/d0/87/415cb472981a8d2e36beeeadf074ebb686cc2bfe8d18de973232da291bd5/pdfplumber-0.11.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 225.2 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20231228 from https://files.pythonhosted.org/packages/eb/9c/e46fe7502b32d7db6af6e36a9105abb93301fa1ec475b5ddcba8b35ae23a/pdfminer.six-20231228-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/25/bd/56d9ec6b9f0fc4e0d95288759f3179f0fcd34b1a1526b75673d2f6d5196f/pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata
     ---------------------------------------- 0.0

In [1]:
import pdfplumber
import re
import pandas as pd

In [3]:
def customer_details(filepath,Password):
    
    text=''
    
    with pdfplumber.open(filepath,password=Password) as pdf:           
        for page in pdf.pages:
            text +=page.extract_text()
    
    pdf_patterns = {'account_number':r'Account\s+Number\s+(\d+)',
               'Account_Holder':r'Name\s+([A-Za-z\s]+)(?=Customer)',
               'Customer/CIF_ID':r'Customer/CIF\s+ID\s+(\d+)',
               'City':r'City\s+([A-Za-z]+)?',
               'State':r'State\s+([A-Za-z]+)?',
               'PinCode':r'Pincode\s+(\d+)',
               'Account_Type':r'Account\s+Type\s+([A-Za-z]+)\s+Account',
               'Mobile_Number':r'Mobile\s+No\s+(\d+)',
               'Email_id':r'Email\s+Id\s+([a-zA-z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA_Z]{2,4})',
               'Home_Branch':r'Home\s+branch\s+([A-Za-z\s]+)(?=\n|$)',
                'IFSC':r'IFSC\s+([A-Za-z\s]+(\d+))'}
    
    ## Customer Details
    
    customer_details = {}
    
    for name,pattern in pdf_patterns.items():
        
        match= re.search(pattern,text)
        
        if name in ('account_number','Customer/CIF_ID','Mobile_Number') :
            
            customer_details[name] = int(match.group(1))
            
        else:
            
            customer_details[name] = match.group(1)
            
    details = pd.DataFrame(customer_details,index=[0])
    
    return details,text



            

In [4]:
def customer_transaction_details(text):

    
    ### Traction data 
    lines = text.splitlines()
    
   
    transactions = []

    header_found = False
    for line in lines:
        if "S.No Date Transaction Id Remarks Amount(Rs.) Balance(Rs.)" in line:
            header_found = True
            continue  
        
        if header_found:
            
            match = re.match(r'(\d+)\s+(\d{2}/\d{2}/\d{4})\s+(\S+)\s+(.+?)\s+([\d,]+\.\d{2})\s+\((Cr|Dr)\)\s+([\d,]+\.\d{2})\s+\((Cr|Dr)\)', line)
            if match:
             
                transaction = {
                    "Account_Number": int(re.search(r'Account\s+Number\s+(\d+)',text).group(1)),
                    "S.No": match.group(1),
                    "Date": match.group(2),
                    "Transaction Id": match.group(3),
                    "Remarks": match.group(4),
                    "Amount (Rs.)": match.group(5)+match.group(6),
                    "Balance (Rs.)": match.group(7)+match.group(8),
                   
                }
                transactions.append(transaction)
                
    transaction_details = pd.DataFrame(transactions)
    return transaction_details


In [5]:
df,pdf_text = customer_details('Union Bank_Statement_Pass-RAJK2006.pdf','RAJK2006')


In [6]:
df

,account_number,Account_Holder,Customer/CIF_ID,City,State,PinCode,Account_Type,Mobile_Number,Email_id,Home_Branch,IFSC
0,583002010001601,RAJKIRAN BHAURAO MISAL,208435581,PUNE,MAHARASHTRA,412201,Savings,917498515126,misalraj@gmail.com,LONI KALBHOR,UBIN0558303


In [7]:
transaction_details = customer_transaction_details(pdf_text)
transaction_details

,Account_Number,S.No,Date,Transaction Id,Remarks,Amount (Rs.),Balance (Rs.)
0,583002010001601,1,04/01/2024,S33381674,UPIAB/437043183865/CR/ZAHENA N/UBIN,480.00Cr,6273.31Cr
1,583002010001601,2,04/01/2024,S95999969,UPIAB/437030685140/CR/SACHIN R/UBIN,2130.00Cr,5793.31Cr
2,583002010001601,3,03/01/2024,S91826775,UPIAB/436999439403/CR/BURHANUD/PUNB,655.00Cr,3663.31Cr
3,583002010001601,4,03/01/2024,S90256830,UPIAB/436959153701/CR/ZAHENA N/UBIN,504.00Cr,3008.31Cr
4,583002010001601,5,03/01/2024,S83044204,UPIAB/400398133155/CR/Chotaram/MAHG,2500.00Cr,2504.31Cr
...,...,...,...,...,...,...,...
1567,583002010001601,1568,08/07/2023,S16456401,UPIAB/318922541616/CR/VIKRAM S/UTIB,3000.00Cr,15837.60Cr
1568,583002010001601,1569,08/07/2023,S14701291,UPIAR/318935506359/DR/EURONETG/ICIC,239.00Dr,12837.60Cr
1569,583002010001601,1570,08/07/2023,S14666819,UPIAR/318935464972/DR/billdesk/ICIC/billdesk.r...,155.00Dr,13076.60Cr
1570,583002010001601,1571,08/07/2023,S12563252,UPIAB/355513543006/CR/ROSHAN R/BKID,500.00Cr,13231.60Cr
